In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

num_samples = 126 # data sample count
time_steps = 200  # Number of time steps in each sample
num_sensors = 6   # Number of sensors

# Initialize an empty list to store data for each channel
input_data = []
input_labels = []

# Define the full file paths for your files
data_files = [
    'sensor1_data.txt',
    'sensor2_data.txt',
    'sensor3_data.txt',
    'sensor4_data.txt',
    'sensor5_data.txt',
    'sensor6_data.txt'
]

label_file = "label_data.txt"

In [2]:
# Load data from each text file and append it to channel_data
for filename in data_files:
    with open(filename, 'r') as file:
        lines = file.readlines()
        channel_values = [list(map(float, line.strip().split())) for line in lines]
        input_data.append(channel_values)

x = np.array(input_data).transpose(1, 2, 0)

# Load labels from the label file
with open(label_file, 'r') as file:
    lines = file.readlines()
    input_labels = [int(value) for line in lines for value in line.strip().split()]

y_labels = np.array(input_labels)

y = to_categorical(y_labels-1,num_classes=2)  # One-hot encode the numerical labels

In [3]:
# check the data in right shape
print(x.shape)
print(y_labels.shape)

(126, 200, 6)
(126,)


In [4]:
# Split the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [5]:
# Building the model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(time_steps, num_sensors)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))  # 2 different people

# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [6]:
# Training the model
model.fit(x_train, y_train, epochs=20, batch_size=2, validation_split=0.2)

Epoch 1/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5169 - loss: 27.4962 - val_accuracy: 0.6500 - val_loss: 6.1284
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6671 - loss: 19.4139 - val_accuracy: 0.7000 - val_loss: 2.8782
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5636 - loss: 12.2029 - val_accuracy: 0.4500 - val_loss: 9.9322
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6709 - loss: 6.4351 - val_accuracy: 0.5500 - val_loss: 5.0643
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6844 - loss: 7.8398 - val_accuracy: 0.7500 - val_loss: 3.9198
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7204 - loss: 4.4006 - val_accuracy: 0.5000 - val_loss: 5.7397
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7592 - loss: 2.5098 - val_accuracy: 0.6000 - val_loss: 2.9745
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8026 - loss: 2.1654 - val_accuracy: 0.6500 - val_l

In [14]:
loss, accuracy = model.evaluate(x_test, y_test)
y_pred = model.predict(x_test)
y_pred_binary = np.argmax(y_pred, axis=1)
y_true_binary = np.argmax(y_test, axis=1)
print(y_true_binary)
print(y_pred_binary)

print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.8846 - loss: 0.5344
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
[1 0 1 1 1 1 0 0 1 0 0 1 0 1 1 0 1 0 0 0 0 0 1 0 0 0]
[1 0 0 0 1 1 1 0 1 0 0 1 0 1 1 0 1 0 0 0 0 0 1 0 0 0]
Test Loss: 0.5344, Test Accuracy: 0.8846
